In [1]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

2024-07-27 19:53:31.347824: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
import pandas as pd
import numpy as np

data = pd.read_csv('./data/gpascore.csv')

data.dropna(inplace=True)

y_data = data['admit']

y_data = np.array(y_data)

x_data = []

for i, rows in data.iterrows():
    x_data.append([ 
        rows['gre'], 
        rows['gpa'], 
        rows['rank'] 
    ])

x_data = np.array(x_data)

print(data.describe())

            admit         gre         gpa       rank
count  425.000000  425.000000  425.000000  425.00000
mean     0.472941  594.329412    3.385929    2.48000
std      0.499856  118.928331    0.396013    0.94928
min      0.000000  220.000000    2.120000    1.00000
25%      0.000000  520.000000    3.120000    2.00000
50%      0.000000  600.000000    3.390000    2.00000
75%      1.000000  680.000000    3.700000    3.00000
max      1.000000  800.000000    4.000000    4.00000


In [3]:
def build_model(hp):
    inputs = tf.keras.Input(shape=3)
    units = hp.Int('units_1', min_value=16, max_value=128, step=16)
    x = tf.keras.layers.Dense(units, activation='relu')(inputs)
    units = hp.Int('units_2', min_value=32, max_value=512, step=16)
    x = tf.keras.layers.Dense(units, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    lr = hp.Float('lr', min_value=1e-5, max_value=1e-2, sampling='log')
    
    optimizer = hp.Choice('optimizer', values=['rmsprop', 'adam'])
    
    if optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=lr)
    elif optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    
    model = tf.keras.Model(inputs, outputs, name='GPA_MODEL')
    
    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model 
    

In [11]:
import keras_tuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='hp_logs',
    project_name='gpa_predict'
)


Reloading Tuner from hp_logs/gpa_predict/tuner0.json


In [5]:
from datetime import datetime


def tensorboard(model_name):
    log_dir = "./tf_logs/fit/"+ str(model_name) + datetime.now().strftime("%Y%m%d-%H%M%S") 
    return tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min'
)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.33,
    random_state=42
)

tuner.search(
    x_train,
    y_train,
    batch_size=64,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[
        tensorboard(tuner.project_name),
        es
    ]
)

Trial 1436 Complete [00h 05m 23s]
val_accuracy: 0.5815602540969849

Best val_accuracy So Far: 0.5815602540969849
Total elapsed time: 01h 49m 48s

Search: Running Trial #1437

Value             |Best Value So Far |Hyperparameter
80                |112               |units_1
480               |96                |units_2
1.0719e-05        |0.0033512         |lr
adam              |adam              |optimizer
5                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
5                 |6                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/5
1/5 [=====>........................] - ETA: 17:02 - loss: nan - accuracy: 0.5156

In [7]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get("units_1"))
print(best_hps.get("units_2"))
print(best_hps.get("optimizer"))

64
128
adam


In [10]:
model = tuner.hypermodel.build(best_hps)
model.fit(
    x_train,
    y_train,
    epochs=50,
    callbacks=[
        es,
        tensorboard(model.name)
    ],
    validation_data=(x_test, y_test)
)

Epoch 1/50
9/9 [==============================] - 2s 34ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5816
Epoch 2/50
9/9 [==============================] - 0s 18ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5816
Epoch 3/50
9/9 [==============================] - 0s 19ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5816
Epoch 4/50
9/9 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5816
Epoch 5/50
9/9 [==============================] - 0s 17ms/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5816
